In [1]:
import pandas as pd
import numpy as np
import os
import gensim
import random
from sklearn.feature_extraction.text import CountVectorizer
from gensim.matutils import Sparse2Corpus
from gensim.models.ldamodel import LdaModel
from gensim import models, similarities
from gensim.corpora import Dictionary
from gensim.models import TfidfModel
import re
import gensim.corpora as corpora
from wordcloud import WordCloud
from gensim.utils import simple_preprocess
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()
import pickle
from pprint import pprint
import matplotlib.pyplot as plt
import re

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/salvatoreesposito/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/usr/local/anaconda3/envs/dme/lib/python3.8/site-packages/ipykernel/pylab/config.py:70: DeprecationWarning: InlineBackend._figure_formats_changed is deprecated in traitlets 4.1: use @observe and @unobserve instead.
  def _figure_formats_changed(self, name, old, new):


In [2]:
data = pd.read_csv('dataset/recipes/recipes-mallet_new.txt', names=('index', 'cuisine', 'ingredients'), delimiter='|')


/usr/local/anaconda3/envs/dme/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:

def treat_ingredients(ing_list):
    output = []
    for ingredient in ing_list:
        ingredient_list = ingredient.split(' ')
        output.append("_".join(ingredient_list))
    return output

/usr/local/anaconda3/envs/dme/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:

def get_similarity(lda, query_vector):
    index = similarities.MatrixSimilarity(lda[corpus])
    sims = index[query_vector]
    return sims

/usr/local/anaconda3/envs/dme/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [44]:
def train_lda_model(ingredients,num_topics = 300,passes = 15,random_state = 0):
    # get list of ingredients transformed
    ingredients_all = ingredients.apply(lambda x: treat_ingredients(x))
    
    #build dict (for Gensim vectorizer)
    dictionary = Dictionary([ing for ing in list(ingredients_all)])
    
    # build corpus: BOW
    corpus = [dictionary.doc2bow(text) for text in list(ingredients_all)]
    
    #train lda
    ldamodel = LdaModel(corpus,num_topics = num_topics, passes = passes,random_state = random_state, id2word = dictionary)
    return ldamodel,dictionary,corpus

/usr/local/anaconda3/envs/dme/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [45]:
# define treat input function, returning a list of tokenized ingredients
def treat_words (words):
    list_words = words.split(",")
    output = []
    for w in list_words:
        output.append("_".join(w.strip().split(" ")))
    return output

/usr/local/anaconda3/envs/dme/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [46]:
def calculate_similarity(query,ldamodel,dct):
    # treat input words
    words_bow = dct.doc2bow(treat_words(query))
    query_vector = ldamodel[words_bow]
    
    #calculate ranking
    sim_rank = get_similarity(lda = ldamodel, query_vector = query_vector)
    sim_rank = sorted(enumerate(sim_rank), key=lambda item: -item[1])
    
    return sim_rank

/usr/local/anaconda3/envs/dme/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [47]:
def calculate_recommendation(sim_rank,groups,n_reco = 10):
    results = [sim_rank[0][0]]
    results_prob = [sim_rank[0][1]]
    result_group = [sim_rank[0][1]]
        
    for recipe,group in zip(sim_rank[1:],groups[1:]):
        if group not in set(result_group):
            results.append(recipe[0])
            result_group.append(group)
            results_prob.append(recipe[1])
        if len(results) == n_reco:
            break
    print(result_group,"\n",results_prob)
    return results

/usr/local/anaconda3/envs/dme/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [34]:
# this is a wrapper function for calculate simu and calculate reco
def get_similarity_reco (query,ldamodel,dct,corpus,n_reco = 10):
    #calculate rank
    sim_rank = calculate_similarity(query,ldamodel,dct)
    #find groups according to lda model
    groups = []
    for l in ldamodel[corpus]:
        try:
            groups.append(l[0][0])
        except:
            groups.append(random.randint(1, 100))
            
    return calculate_recommendation(sim_rank,groups,n_reco)

/usr/local/anaconda3/envs/dme/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [48]:
def print_reco(results):
    return data.iloc[results]

/usr/local/anaconda3/envs/dme/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [49]:
def pretty_name (name):
    return " ".join([ word.capitalize() for word in name.split(" ") if word != ""])

/usr/local/anaconda3/envs/dme/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [50]:
ldamodel,dictionary,corpus = train_lda_model(data.ingredients)


/usr/local/anaconda3/envs/dme/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [51]:
query = 'apple,cabbage,cider vinegar'


/usr/local/anaconda3/envs/dme/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [52]:
results = get_similarity_reco (query, ldamodel, dct = dictionary, corpus = corpus,n_reco = 1)


/usr/local/anaconda3/envs/dme/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[0.0, 22, 34, 1, 64, 77, 52, 60, 125, 47, 92, 62, 56, 50, 55, 67, 19, 100, 32, 6, 68, 70, 23, 182, 44, 115, 36, 25, 71, 66, 43, 274, 263, 161, 30, 153, 105, 131, 86, 8, 97, 165, 80, 117, 83, 150, 149, 211, 227, 128, 140, 157, 183, 63, 111, 245, 237, 172, 135, 228, 259, 264, 79, 167, 168, 41, 203, 107, 223, 270, 238, 251, 257, 205, 255, 262, 180, 246, 159, 210, 222, 156, 217, 132, 277, 154, 267, 241, 31, 294, 191, 265, 269, 209, 184, 283] 
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.

In [53]:
print_reco(results)


/usr/local/anaconda3/envs/dme/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,index,cuisine,ingredients
0,1,1,"broth,chinese cabbage,cornstarch,egg,ginger,gr..."
1,2,1,"broccoli,cornstarch,garlic,ginger,green onion,..."
2,3,1,"beef,broccoli,cornstarch,garlic,ginger,peanut ..."
3,4,1,"bamboo shoot,bean sprouts,celery,chicken,egg,m..."
5,6,1,"chicken,cornstarch,dried chile pepper,egg,garl..."
...,...,...,...
3770,3771,11,"cod,garlic,green onion,olive oil,sweet pepper"
3932,3933,12,"basil,coconut milk or cream,curry paste,fish s..."
3972,3973,12,"chile pepper,coconut milk or cream,curry paste..."
4067,4068,12,"bean sprouts,chinese chives,dried chile pepper..."


In [54]:
lda_25,dict_25,corpus_25 = train_lda_model(data.ingredients,num_topics = 25)


/usr/local/anaconda3/envs/dme/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [55]:
results_25 = get_similarity_reco (query, lda_25, dct = dict_25, corpus = corpus_25,n_reco = 10)


/usr/local/anaconda3/envs/dme/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


IndexError: index 24 is out of bounds for axis 0 with size 24

In [ ]:
print_reco(results_25)


In [ ]:
ldamodel.top_topics(corpus)
